In [ ]:
# Instalar dependências
!pip install flask yt-dlp
!apt install ffmpeg -y
!wget -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared

# Criar servidor Flask
import os
import subprocess
import yt_dlp
from flask import Flask, request, send_file

app = Flask(__name__)

DOWNLOAD_FOLDER = "downloads"
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

HTML_PAGE = """
<!DOCTYPE html>
<html lang="pt">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Baixar MP3 do YouTube</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            margin: 0;
            padding: 20px;
            background-color: #121212;
            color: white;
        }
        h2 {
            margin-bottom: 20px;
        }
        input {
            padding: 10px;
            width: 80%;
            max-width: 400px;
            font-size: 16px;
            border-radius: 5px;
            border: none;
        }
        button {
            background-color: red;
            color: white;
            border: none;
            padding: 10px 15px;
            font-size: 16px;
            cursor: pointer;
            margin-top: 10px;
            border-radius: 5px;
        }
        button:hover {
            background-color: darkred;
        }
        #pasteBtn {
            background-color: grey;
        }
        #pasteBtn:hover {
            background-color: darkgrey;
        }
        #clearBtn {
            background-color: blue;
        }
        #clearBtn:hover {
            background-color: darkblue;
        }
    </style>
</head>
<body>
    <h2>Baixar MP3 do YouTube</h2>
    <form action="/download" method="post" onsubmit="formatURL()">
        <input type="text" id="url" name="url" placeholder="Cole a URL do YouTube" required>
        <br>
        <button type="button" id="pasteBtn">Colar</button>
        <button type="button" id="clearBtn">Limpar</button>
        <button type="submit">Baixar MP3</button>
    </form>
    <script>
        document.getElementById("pasteBtn").addEventListener("click", async function() {
            try {
                const text = await navigator.clipboard.readText();
                document.getElementById("url").value = text;
            } catch (err) {
                alert("Erro ao acessar o clipboard");
            }
        });

        document.getElementById("clearBtn").addEventListener("click", function() {
            document.getElementById("url").value = "";
        });

        function formatURL() {
            let url = document.getElementById("url").value;
            if (url.includes("youtu.be/")) {
                let videoID = url.split("youtu.be/")[1].split("?")[0];
                document.getElementById("url").value = "https://www.youtube.com/watch?v=" + videoID;
            }
        }
    </script>
</body>
</html>
"""

@app.route('/')
def home():
    return HTML_PAGE

@app.route('/download', methods=['POST'])
def download():
    url = request.form.get('url')
    if not url:
        return "Erro: Nenhuma URL fornecida."

    try:
        ydl_opts = {"quiet": True, "extract_flat": True}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)

        title = info.get("title", "Desconhecido")
        author = info.get("uploader", "Desconhecido")
        safe_title = "".join(c for c in f"{author} - {title}" if c.isalnum() or c in " _-").strip()
        file_name = os.path.join(DOWNLOAD_FOLDER, f"{safe_title}.mp3")

        # Adicionando a qualidade de áudio 320kbps
        command = ["yt-dlp", "-x", "--audio-format", "mp3", "--audio-quality", "320K", "-o", file_name, url]
        subprocess.run(command, check=True)

        return f'<h3>Download concluído! <a href="/file?name={safe_title}.mp3">Clique aqui para baixar</a></h3> <br> <a href="/">Voltar</a>'
    except Exception as e:
        return f"Erro ao baixar: {str(e)}"

@app.route('/file')
def get_file():
    name = request.args.get("name", "")
    file_path = os.path.join(DOWNLOAD_FOLDER, name)

    if os.path.exists(file_path):
        return send_file(file_path, as_attachment=True)
    return "Arquivo não encontrado."

# Iniciar o servidor Flask e Cloudflared
def start_server():
    from threading import Thread
    try:
        Thread(target=lambda: app.run(host='0.0.0.0', port=80)).start()
    except:
        pass

start_server()

!./cloudflared tunnel --url http://127.0.0.1:80 --no-autoupdate
